# Extraction of resonances from GNDS

Use this `Notebook` to pull out all of the resonance data from the XML-formatted GNDS representations of the ENDF-B/VIII.1 library.  Change explicit definitions of path names accordingly.

In [1]:
from fudge import map as fudgeMap

In [2]:
ENDF81 = fudgeMap.read("/Users/davidmatters/ENDF/ENDF-B-VIII.1-GNDS/neutrons.map")

In [3]:
# Create list of neutron targets from XML ENDF files
import glob
ENDF_XML_PATH = '/Users/davidmatters/ENDF/ENDF-B-VIII.1-GNDS/neutrons'
xml_list = [x for x in glob.glob("%s/*.xml"%ENDF_XML_PATH)]
n_targets = []
for xml in xml_list:
    xml_file = xml.split('neutrons/')[1]
    Z = int(xml_file.split('n-')[1].split('_')[0])
    element = xml_file.split('_')[1]
    A = None
    try:
        A = int(xml_file.split('_')[2].split('.')[0])
    except ValueError:
        A = xml_file.split('_')[2].split('.')[0]
        if 'm' in A:
            if A[0]=='0' and A[1]=='0':
                A = A.strip(A[0:2])
            elif A[0]=='0':
                A = A.strip(A[0:1])
            else:
                pass
            A = A.replace('m','_m')
                    
    n_targets.append(element+str(A))
n_targets = sorted(n_targets)
print("List object contains {0} n-reaction targets from XML-formatted ENDF libraries.\n".format(len(n_targets)))
print(n_targets)

List object contains 558 n-reaction targets from XML-formatted ENDF libraries.

['Ac225', 'Ac226', 'Ac227', 'Ag107', 'Ag108', 'Ag109', 'Ag110_m1', 'Ag111', 'Ag112', 'Ag113', 'Ag114', 'Ag115', 'Ag116', 'Ag117', 'Ag118_m1', 'Al26_m1', 'Al27', 'Am240', 'Am241', 'Am242', 'Am242_m1', 'Am243', 'Am244', 'Am244_m1', 'Ar36', 'Ar37', 'Ar38', 'Ar39', 'Ar40', 'Ar41', 'As73', 'As74', 'As75', 'Au197', 'B10', 'B11', 'Ba130', 'Ba131', 'Ba132', 'Ba133', 'Ba134', 'Ba135', 'Ba136', 'Ba137', 'Ba138', 'Ba139', 'Ba140', 'Be7', 'Be9', 'Bi209', 'Bi210_m1', 'Bk245', 'Bk246', 'Bk247', 'Bk248', 'Bk249', 'Bk250', 'Br79', 'Br80', 'Br81', 'C12', 'C13', 'Ca40', 'Ca41', 'Ca42', 'Ca43', 'Ca44', 'Ca45', 'Ca46', 'Ca47', 'Ca48', 'Cd106', 'Cd107', 'Cd108', 'Cd109', 'Cd110', 'Cd111', 'Cd112', 'Cd113', 'Cd114', 'Cd115_m1', 'Cd116', 'Ce136', 'Ce137', 'Ce137_m1', 'Ce138', 'Ce139', 'Ce140', 'Ce141', 'Ce142', 'Ce143', 'Ce144', 'Cf246', 'Cf247', 'Cf248', 'Cf249', 'Cf250', 'Cf251', 'Cf252', 'Cf253', 'Cf254', 'Cl35', 'Cl36', 'Cl37

In [4]:
# Do a test loop over all of the Si data
silicon = ['Si28','Si29','Si30','Si31','Si32']

In [7]:
# Loop over all targets and extract Reich-Moore and Breit-Wigner-stored resonance parameters as appropriate.
import pandas as pd

TARGETDIR_BREIT_WIGNER="extraction-gnds/BreitWigner"
TARGETDIR_REICH_MOORE="extraction-gnds/ReichMoore"

# Uncomment following line to test data extraction on Si data only
#for t in silicon:
# Uncomment the following line to loop over all targets
for t in n_targets:
    if t!='n1':
        nt = ENDF81.find(projectile="n", target=t).read()
        try:
            RRR = nt.resonances.resolved.evaluated
            #if 'BreitWigner' in str(RRR):
            if RRR.approximation.value == 'MultiLevel':
                print("{0}: {1}".format(t,type(RRR)))
                
                columns = [column.name for column in RRR.resonanceParameters.table.columns]
                df_BW = pd.DataFrame(RRR.resonanceParameters.table.data, columns=columns)
                df_BW.to_csv("{0}/n-res-{1}.csv".format(TARGETDIR_BREIT_WIGNER,t), index=False)
            elif RRR.approximation.value == 'ReichMoore':
                print("{0}: {1}".format(t,type(RRR)))
                
                spin_data = []
                elasticReaction = nt.getReaction("elastic") # Need for L-value
                for spinGroup in RRR.spinGroups:
                    spin = eval(f"{spinGroup.spin}")
                    parity = int(f"{spinGroup.parity}")
                    num_res = len(spinGroup.resonanceParameters.table)
                    #spin_data.append([spin, parity, num_res])
                    
                    # Find L-values
                    elasticChannel = [channel for channel in spinGroup.channels if channel.resonanceReaction == elasticReaction.label]
                    try:
                        assert len(elasticChannel) == 1
                        #print("L=",elasticChannel[0].L)
                        L = elasticChannel[0].L
                        spin_data.append([spin, parity, L, num_res])
                    except AssertionError:
                        print("L-PROBLEM: ",len(elasticChannel))
                        L = elasticChannel[0].L
                        spin_data.append([spin, parity, L, num_res])
                    
                spin_col = []
                parity_col = []
                L_col = []
                # I don't really like this but it appears to generate the correct J and Pi correspondence
                for i,sd in enumerate(spin_data):
                    spin_col += [sd[0]]*sd[3]
                    parity_col += [sd[1]]*sd[3]
                    L_col += [sd[2]]*sd[3]
                
                # Create column labels
                capture = nt.getReaction("capture")
                captureLabel = capture.label + " width"
                elastic = nt.getReaction("elastic")
                elasticLabel = elastic.label + " width"
                spinLabel = "J"
                parityLabel = "Pi"
                LLabel = "L"
                
                parameters = {"energy": [], captureLabel: [], elasticLabel: [], 
                              spinLabel: spin_col, parityLabel: parity_col, LLabel: L_col}
                for spinGroup in RRR.spinGroups:
                    for label in ("energy", captureLabel, elasticLabel):
                        parameters[label] += spinGroup.resonanceParameters.table.getColumn(label)
                df_RM = pd.DataFrame(parameters)
                df_RM.to_csv("{0}/n-res-{1}.csv".format(TARGETDIR_REICH_MOORE,t), index=False)
            else:
                print(type(RRR), "Resonances not handled.")
        except AttributeError:
            print("{0}: No neutron resonance parameters.".format(t))

Ac225: No neutron resonance parameters.
Ac226: No neutron resonance parameters.
Ac227: No neutron resonance parameters.
Ag107: <class 'fudge.resonances.resolved.BreitWigner'>
Ag108: <class 'fudge.resonances.resolved.BreitWigner'>
Ag109: <class 'fudge.resonances.resolved.BreitWigner'>
Ag110_m1: <class 'fudge.resonances.resolved.BreitWigner'>
Ag111: <class 'fudge.resonances.resolved.BreitWigner'>
Ag112: <class 'fudge.resonances.resolved.BreitWigner'>
Ag113: <class 'fudge.resonances.resolved.BreitWigner'>
Ag114: <class 'fudge.resonances.resolved.BreitWigner'>
Ag115: <class 'fudge.resonances.resolved.BreitWigner'>
Ag116: <class 'fudge.resonances.resolved.BreitWigner'>
Ag117: <class 'fudge.resonances.resolved.BreitWigner'>
Ag118_m1: <class 'fudge.resonances.resolved.BreitWigner'>
Al26_m1: <class 'fudge.resonances.resolved.BreitWigner'>
Al27: <class 'fudge.resonances.resolved.RMatrix'>
Am240: No neutron resonance parameters.
Am241: <class 'fudge.resonances.resolved.BreitWigner'>
Am242: <clas